In [ ]:
#Tuodaan PuLP kirjasto
from pulp import *
# Jos tulee virheilmoitus niin anna ensin komento "pip install pulp"

#Data
tuotenimet=["A","B","C","Cp","D","Dp","E","F","Fp","G","H","A nt","B nt","C nt","Cp nt","D nt","Dp nt","F nt","Fp nt","G nt","H nt","A 2.g","B 2.g","C 2.g","D 2.g","E 2.g","F 2.g","G 2.g","H 2.g","nt"]
myynnit=[372,393,56,50,26,46,86,222,284,94,345,451,111,81,37,50,103,132,86,244,726,310,150,92,210,256,210,380,298,0]
kustannukset=[25,12.5,12.5,5,12.5,5,25,12.5,5,7.5,25,12.5,12.5,5,12.5,12.5,7.5,12.5,5,50,50,37.5,37.5,37.5,50,32.5,30,100,50,50]
budjetti = 300

tuoteindeksit = range(len(tuotenimet))

#Luodaan (maksimoiva) lineaarinen optimointi malli PuLP:ssa
malli = LpProblem("Tuoteportdoliomalli", LpMaximize)

#Luodaan binääriset päätösmuuttujat x_i: x_i = 1 jos tuote kehitetään, muuten x_i=0
#Kategoria on LpInteger, eli kokonaisluku, jonka ylä- ja alarajat ovat 1 ja 0 -> Vain 0 ja 1 sallittuja arvoja
x={}
for i in tuoteindeksit:
    x[i] = LpVariable("Kehitetään_"+tuotenimet[i]+"?", lowBound=0, upBound=1, cat=LpInteger)

#Lisätään kohdefunktio eli summa kaikkien reittien kuljetuskustannuksista
malli += (lpSum([myynnit[i]*x[i] for i in tuoteindeksit]), "Kokonaismyynti")

#Lisätään budjettirajoite 
------------

#Lisätään loogiset rajoitteet
for i in tuoteindeksit:
    for j in tuoteindeksit:
        if i!=j:
            #Plustuotteet
            if (tuotenimet[i]+"p")==tuotenimet[j]:
                malli += (x[i]>=x[j],"Ensin"+tuotenimet[i]+", sitten "+tuotenimet[j])
            #2. generaation tuotteet
            if (tuotenimet[i]+" 2.g")==tuotenimet[j]:
                malli += (x[i]>=x[j],"Ensin"+tuotenimet[i]+", sitten "+tuotenimet[j])
            #Uutta teknologiaa hyödyntävät tuotteet
            if (tuotenimet[i]+" nt")==tuotenimet[j]:
                malli += (x[i]>=x[j],"Ensin"+tuotenimet[i]+", sitten "+tuotenimet[j])
                malli += (x[29]>=x[j],"Ensin"+tuotenimet[i]+", sitten "+tuotenimet[29])

#Ratkaistaan malli, eli etsitään optimaalinen ratkaisu (tai todetaan että mallille ei ole käypää ratkaisua)     
#malli.solve()
#Käytä tätä komentoa, jos et halua ratkaisimen näyttävän tietoa ratkaisemisesta
malli.solve(PULP_CBC_CMD(msg=False))   


# Tulostetaan tuotteet, jotka kuuluvat optimaaliseen portfolioon
kokonaismyynti = 0
kokonaiskustannus = 0 
print("Optimaaliset tuotteet:")
for i in tuoteindeksit:
    kokonaismyynti = kokonaismyynti + myynnit[i]*x[i].varValue
    kokonaiskustannus = kokonaiskustannus + kustannukset[i]*x[i].varValue
    if x[i].varValue==1:
        print(tuotenimet[i])
    
# Tulostetaan optimaalinen kohdefunktion arvo 
print("Kokonaismyynti = %s " % kokonaismyynti)
print("Kokonaiskustannus = %s " % kokonaiskustannus)



